In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome()
driver.get("https://www.google.com/maps")

# Search for software houses in Johar Town Lahore
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@id='searchboxinput']"))
)
search_box.send_keys("software houses in Johar Town Lahore")
search_box.send_keys(Keys.RETURN)
time.sleep(5)  # Wait for results to load

# Initialize lists to store extracted data
names = []
addresses = []
phone_numbers = []

# Scroll and extract data from all results
previous_results_count = 0
max_attempts = 10  # Maximum attempts to scroll without new results
attempts = 0

while attempts < max_attempts:
    try:
        # Locate all result objects on the page
        results = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'hfpxzc')]"))
        )

        # If new results are found, reset the attempt counter
        if len(results) > previous_results_count:
            attempts = 0  # Reset attempts since new results are loaded
        else:
            attempts += 1  # Increment attempts since no new results are loaded

        # Extract details from all results after scrolling
        for result in results[previous_results_count:]:  # Process only new results
            try:
                # Click on the result to open details
                driver.execute_script("arguments[0].click();", result)
                time.sleep(5)  # Wait for the details page to load

                # Extract name
                name = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, "//h1[contains(@class, 'DUwDvf')]"))
                ).text

                # Extract address
                address = driver.find_element(By.XPATH, "//button[@data-item-id='address']").text

                # Extract phone number
                phone_number = driver.find_element(By.XPATH, "//button[@data-tooltip='Copy phone number']").text

                # Append extracted data to lists
                names.append(name)
                addresses.append(address)
                phone_numbers.append(phone_number)

                print(f"Extracted: Name={name}, Address={address}, Phone={phone_number}")
            except Exception as e:
                print(f"Error extracting details for a result: {e}")
            
            # # Go back to results list
            # driver.execute_script("window.history.go(-1)")
            # time.sleep(5)  # Wait for the results page to reload

        previous_results_count = len(results)  # Update the count of processed results

        # Scroll to the last result
        driver.execute_script("arguments[0].scrollIntoView(true);", results[-1])
        time.sleep(5)  # Allow time for new results to load

    except Exception as e:
        print(f"Error during scrolling or processing results: {e}")
        break

# Save the data to a DataFrame
data = pd.DataFrame({
    "Name": names,
    "Address": addresses,
    "Phone Number": phone_numbers
})

# Save to Excel or CSV
data.to_excel("software_houses_data.xlsx", index=False)
print("Data saved to 'software_houses_data.xlsx'")

# Close the browser
driver.quit()



